In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import distance_transform_edt


In [2]:
data = pd.read_csv('data/pok_2010_2025.csv')
data.head()

,latitude,longitude,date,species,gear,weight,pok_ratio,pok
0,65.0000,-24.1333,2010-01-01,"['HAD', 'COD']",LLN,"[2200.0, 200.0]",0.0,0
1,65.0667,-23.9000,2010-01-02,"['COD', 'HAD']",LLN,"[2000.0, 10000.0]",0.0,0
2,64.2500,-22.2500,2010-01-02,"['CAA', 'COD', 'HAD']",LLN,"[300.0, 1700.0, 800.0]",0.0,0
3,66.3500,-23.9667,2010-01-02,"['COD', 'HAD']",LLN,"[2300.0, 700.0]",0.0,0
4,66.0167,-21.0000,2010-01-02,"['HAD', 'CAS', 'COD']",LLN,"[3500.0, 10.0, 1700.0]",0.0,0


In [3]:
bathy = xr.open_dataset('rawdata/GEBCO_15_Oct_2025/gebco_2025_n68.0_s61.0_w-30.0_e-10.0.nc')

# add depth to data from bathymetry elevation
data['depth'] = bathy.interp(lon=('points', data['longitude']), lat=('points', data['latitude']))['elevation'].values

# remove rows with depth > 0 (land)
data = data[data['depth'] <= 0]

# reverse sign so positive depth values
data['depth'] = -data['depth']

In [4]:
def nearest_non_nan(da, lons, lats):
    """
    For a 2D DataArray (latitude x longitude), find nearest non-NaN values
    at given (lon, lat) coordinates.
    """
    lon_grid, lat_grid = np.meshgrid(da.longitude.values, da.latitude.values)
    mask = np.isnan(da.values)
    # Distance to nearest non-NaN cell + indices of nearest valid cells
    dist, (idx_lat, idx_lon) = distance_transform_edt(mask, return_indices=True)
    # Map indices to coordinates
    nearest_values = da.values[idx_lat, idx_lon]
    # Interpolate per requested coordinate
    nearest_points = []
    for lo, la in zip(lons, lats):
        # find nearest grid index
        j = np.abs(da.longitude.values - lo).argmin()
        i = np.abs(da.latitude.values - la).argmin()
        nearest_points.append(nearest_values[i, j])
    return np.array(nearest_points)

def compute_gradient(temp_slice):
    """
    Calculates the masked gradient magnitude for a 2D xarray
        input has to have 'latitude' and 'longitude' coordinates.
    """
    # Get coordinates and original data
    lats = temp_slice['latitude'].values
    lons = temp_slice['longitude'].values
    temp_data_original = temp_slice.values

    # Create the land mask from original NaNs
    nan_mask = np.isnan(temp_data_original)

    # Fill NaNs using nearest neighbor interpolation
    temp_data_filled = temp_slice.interpolate_na(dim='latitude', method='nearest')
    temp_data_filled = temp_data_filled.interpolate_na(dim='longitude', method='nearest')

    # gradient calculation
    grad_y, grad_x = np.gradient(temp_data_filled, lats, lons, axis=(0, 1))
    grad_magnitude = np.sqrt(grad_y**2 + grad_x**2)
    grad_magnitude[nan_mask] = np.nan # reapply land mask
    grad = xr.DataArray(grad_magnitude, coords=temp_slice.coords, dims=temp_slice.dims)

    return grad

In [ ]:
phy1 = xr.open_dataset('rawdata/copernicus/phy_features_my.nc')
phy2 = xr.open_dataset('rawdata/copernicus/phy_features_myint.nc')
phy_split = pd.to_datetime('2021-06-30')

# add gradient of thetao to both phy1 and phy2
grad_list_phy1 = []
for t_slice in phy1['thetao']:
    grad_list_phy1.append(compute_gradient(t_slice))
phy1['thetao_grad'] = xr.concat(grad_list_phy1, dim='time')

grad_list_phy2 = []
for t_slice in phy2['thetao']:
    grad_list_phy2.append(compute_gradient(t_slice))
phy2['thetao_grad'] = xr.concat(grad_list_phy2, dim='time')


# augment data with phy1 for dates <= 2021-06-30 and with phy2 for dates > 2021-06-30
data['date'] = pd.to_datetime(data['date'])
data1 = data[data['date'] <= phy_split].copy()
data2 = data[data['date'] > phy_split].copy()

# make emty columns for each variable in phy1 
for var in phy1.data_vars:
    data1[var] = np.nan
    data2[var] = np.nan

# interpolate phy1 and phy2 to data1 and data2
for var in phy1.data_vars:
    vals1 = []
    for t in data1['date'].unique():
        da = phy1[var].sel(time=t, method='nearest')
        idx = data1['date'] == t
        vals1.extend(nearest_non_nan(da, data1.loc[idx, 'longitude'], data1.loc[idx, 'latitude']))
    data1.loc[:, var] = vals1

    vals2 = []
    for t in data2['date'].unique():
        da = phy2[var].sel(time=t, method='nearest')
        idx = data2['date'] == t
        vals2.extend(nearest_non_nan(da, data2.loc[idx, 'longitude'], data2.loc[idx, 'latitude']))
    data2.loc[:, var] = vals2

data = pd.concat([data1, data2])

In [ ]:
bio1 = xr.open_dataset('rawdata/copernicus/bio_features_my.nc')
bio2 = xr.open_dataset('rawdata/copernicus/bio_features_myint.nc')
bio_split = pd.to_datetime('2022-12-31')

# augment data with bio1 for dates <= 2022-12-31 and with bio2 for dates > 2022-12-31
data['date'] = pd.to_datetime(data['date'])
data1 = data[data['date'] <= bio_split].copy()
data2 = data[data['date'] > bio_split].copy()

# make emty columns for each variable in bio1
for var in bio1.data_vars:
    data1[var] = np.nan
    data2[var] = np.nan

# interpolate bio1 and bio2 to data1 and data2
for var in bio1.data_vars:
    vals1 = []
    for t in data1['date'].unique():
        da = bio1[var].sel(time=t, method='nearest')
        idx = data1['date'] == t
        vals1.extend(nearest_non_nan(da, data1.loc[idx, 'longitude'], data1.loc[idx, 'latitude']))
    data1.loc[:, var] = vals1

    vals2 = []
    for t in data2['date'].unique():
        da = bio2[var].sel(time=t, method='nearest')
        idx = data2['date'] == t
        vals2.extend(nearest_non_nan(da, data2.loc[idx, 'longitude'], data2.loc[idx, 'latitude']))
    data2.loc[:, var] = vals2

data = pd.concat([data1, data2])

In [22]:
# create cyclical features for day of year
data['day_of_year'] = pd.to_datetime(data['date']).dt.dayofyear
#data['year'] = pd.to_datetime(data['date']).dt.year
data['day_cos'] = np.cos(2 * np.pi * data['day_of_year'] / 365.25)
data['day_sin'] = np.sin(2 * np.pi * data['day_of_year'] / 365.25)

# remove day_of_year column
data = data.drop(columns=['day_of_year'])

data.head()

,latitude,longitude,date,species,gear,weight,pok_ratio,pok,depth,thetao,...,so,thetao_grad,chl,no3,nppv,o2,po4,si,day_cos,day_sin
0,65.0000,-24.1333,2010-01-01,"['HAD', 'COD']",LLN,"[2200.0, 200.0]",0.0,0,252.778,4.817377,...,35.041351,6.683562,0.095110,8.660581,0.013150,292.646240,0.626149,4.391223,0.999852,0.017202
1,65.0667,-23.9000,2010-01-02,"['COD', 'HAD']",LLN,"[2000.0, 10000.0]",0.0,0,166.442,4.473129,...,35.029144,4.666329,0.084030,8.964667,0.022680,286.582703,0.638372,4.373333,0.999408,0.034398
2,64.2500,-22.2500,2010-01-02,"['CAA', 'COD', 'HAD']",LLN,"[300.0, 1700.0, 800.0]",0.0,0,53.250,3.352489,...,34.933010,2.230349,0.173263,7.805847,0.219000,313.892975,0.590805,4.402498,0.999408,0.034398
3,66.3500,-23.9667,2010-01-02,"['COD', 'HAD']",LLN,"[2300.0, 700.0]",0.0,0,121.770,4.462141,...,35.177158,5.423651,0.075653,9.301874,0.018495,304.433960,0.672640,4.809489,0.999408,0.034398
4,66.0167,-21.0000,2010-01-02,"['HAD', 'CAS', 'COD']",LLN,"[3500.0, 10.0, 1700.0]",0.0,0,128.238,4.922117,...,35.180210,0.814602,0.078275,8.722303,0.012088,314.046326,0.652994,4.954331,0.999408,0.034398


In [23]:
# save the full dataset
data.to_csv('data/pok_2010_2025_augmented.csv', index=False)
